<a href="https://colab.research.google.com/github/hvkchaitanya7/grp3capstoneprj/blob/main/ImageCaptionGroup3_Flickr30k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
1)Download FLickr30k dataset
2)Divide the images into 4 folders
3)Upload 4 folders into drive
4)Train model on 4 folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')#,force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Change worling directory to Google drive
import os
os.chdir('/content/drive/MyDrive/Project Files/')

In [ ]:
#List folder in current working directory
os.listdir()

For below two cells,unzip all four folder one after other

In [ ]:
!unzip flickr30k_4.zip

In [ ]:
# Import all the critical libraries including keras 
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import optimizers
from keras import Input, layers
from keras.applications.inception_v3 import preprocess_input
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [ ]:
# from keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
#Read results_flickr30k.csv and retain only first two columns
#We have uploaded this csv into content folder on colab
df_results=pd.read_csv('/content/results_flickr30k.csv')
df_results=df_results[['image_name',' comment']]

df_results.head()

In [ ]:
# creating a dictionary of all the captions for an image 

mapping = dict()
# process lines
for row in (df_results.values):
    
    # take the first token as the image id, the rest as the description
    image_id, image_desc = row[0], row[1]
    # convert description tokens back to string
    #image_desc = ' '.join(image_desc)
    # create the list if needed
    if image_id not in mapping:
        mapping[image_id] = list()
    # store description
    mapping[image_id].append(image_desc)

In [ ]:
len(mapping)

31783

In [ ]:
# creating a dictionary of all the captions for an image 
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping


In [ ]:
# parse descriptions
descriptions = mapping
print('Loaded: %d ' % len(descriptions))

Loaded: 31783 


In [ ]:
descriptions

In [ ]:
#load first five elements of descriptions
list(descriptions.keys())[:5]

['1000092795.jpg',
 '10002456.jpg',
 '1000268201.jpg',
 '1000344755.jpg',
 '1000366164.jpg']

In [ ]:
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			#print(desc)
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [ ]:
!nvidia-smi

Sun Jul  3 06:50:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# convert the loaded descriptions into a vocabulary of words
# Get Unique words from all the descriptions
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 18692


In [ ]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')


In [ ]:
#Get dataset of images for all flickr30k folders
dataset = list()
#Change flickr30k folder name 
for img in os.listdir('/content/drive/MyDrive/Project Files/flickr30k_1'):
    dataset.append(img)
for img in os.listdir('/content/drive/MyDrive/Project Files/flickr30k_2'):
    dataset.append(img)
for img in os.listdir('/content/drive/MyDrive/Project Files/flickr30k_3'):
    dataset.append(img)
for img in os.listdir('/content/drive/MyDrive/Project Files/flickr30k_4'):
    dataset.append(img)
dataset=set(dataset)

In [ ]:
len(dataset) #/content/drive

0

In [ ]:
# Below path contains all the images
images = '/content/drive/MyDrive/Project Files/flickr30k_1/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')
images = '/content/drive/MyDrive/Project Files/flickr30k_2/'
# Create a list of all image names in the directory
img.extend(glob.glob(images + '*.jpg'))
images = '/content/drive/MyDrive/Project Files/flickr30k_3/'
# Create a list of all image names in the directory
img.extend(glob.glob(images + '*.jpg'))
images = '/content/drive/MyDrive/Project Files/flickr30k_4/'
# Create a list of all image names in the directory
img= glob.glob(images + '*.jpg')

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', dataset)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=21532


In [ ]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    #print('after to array', x.shape)
    # Add one more dimension
    x = np.expand_dims(x, axis=0) # this adds one  more dimension for batch size
    #print('after dimension' , x.shape)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [ ]:
# Load the inception v3 model
model_images = InceptionV3(weights='imagenet')

96124928/96112376 [==============================] - 1s 0us/step


In [ ]:
model_images.summary()

In [ ]:
# Create a new model, by removing the last layer (output layer) from the inception v3
# The model is used derive the Feature vector from the pre trained Inceptio V3 and return 2048 size vector for each image
model_new = Model(model_images.input, model_images.layers[-2].output)

In [ ]:
model_new.summary()

In [ ]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    #print('encode' , fea_vec.shape)
    return fea_vec

In [ ]:
encoding_train = {}

In [ ]:
images = '/content/drive/MyDrive/Project Files/flickr30k_1/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [ ]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
start = time()
for img_ in img:
    encoding_train[img_[len(images):]] = encode(img_)
print("Time taken in seconds =", time()-start)

Time taken in seconds = 523.3595967292786


In [ ]:
len(encoding_train),len(img)

(3593, 31783)

In [ ]:
!mkdir '/content/drive/MyDrive/Project Files/Pickle/'

In [ ]:
pkl_train_file="/content/drive/MyDrive/Project Files/Pickle/encoded_train_images.pkl"

In [ ]:
pkl_train_file="https://drive.google.com/file/d/1HugSX1wY8GFFBTfyZTXk8U1JocbxVHs8/view?usp=sharing"

In [ ]:
# Save the bottleneck train features to disk
with open(pkl_train_file, "wb") as encoded_pickle:
    dump(encoding_train, encoded_pickle)

    # /content/sample_data/Pickle/encoded_train_images.pkl

In [ ]:
train_features = load(open(pkl_train_file, "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=21532


In [ ]:
# Create a list of all the training captions
all_train_captions = []
for key, val in descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

158915

In [ ]:
all_train_captions

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 18693 -> 5094


In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [ ]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

5095

In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 70


In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            if key in photos:
                photo = photos[key]#+'.jpg']
                for desc in desc_list:
                    # encode the sequence
                    seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                    # split one sequence into multiple X, y pairs
                    for i in range(1, len(seq)):
                        # split into input and output pair
                        in_seq, out_seq = seq[:i], seq[i]
                        # pad input sequence
                        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                        # encode output sequence
                        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                        # store
                        X1.append(photo)
                        X2.append(in_seq)
                        y.append(out_seq)
                # yield the batch data
                if n==num_photos_per_batch:
                    yield [[array(X1), array(X2)], array(y)]
                    X1, X2, y = list(), list(), list()
                    n=0

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-08-10 13:56:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-08-10 13:56:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-08-10 13:56:37--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:

# Load Glove vectors
glove_dir = '/content/drive/MyDrive/Project Files/'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(5095, 200)

In [ ]:
vocab_size

5095

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256,return_sequences =True)(se2)
decoder1 = add([fe2, se3])
decoder2 = LSTM(256)(decoder1)
decoder3 = Dense(256, activation='relu')(decoder2)
outputs = Dense(vocab_size, activation='softmax')(decoder3)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 70)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 70, 200)      1019000     ['input_3[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_2[0][0]']                
                                                                                            

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
epochs = 1
number_pics_per_bath = 100
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
for i in range(10):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    #model.save('./model_weights/model_flickr30k_1' + str(i) + '.h5')

In [ ]:
model.save('./model_weights/model_flickr30k_final' + '.h5')

In [ ]:
history.history

{'accuracy': [0.4483354389667511],
 'loss': [2.106640577316284],
 'val_accuracy': [0.31414780020713806],
 'val_loss': [3.894533634185791]}

In [ ]:
scores=model.evaluate(train_generator,steps=50)

In [ ]:
model.optimizer.lr = 0.0001
epochs = 15
number_pics_per_bath = 20
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
for i in range(2):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    history=model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    #model.save('./model_weights/model_' + str(i) + '.h5')

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import tensorflow as tf
def get_bleuscore(pred_desc, image_name):
    cand_desc = pred_desc.split()
    #print("blue score desc", cand_desc)
    img_name = image_name.split(".jpg")[0]
    refer_desc = descriptions[img_name]
    refer_list = []
    for s in refer_desc:
         s = s.split()
         refer_list.append(s)
    #print(s)
    print("Expert Caption:",refer_desc)
    #print (refer_list)
    score = sentence_bleu(refer_list,cand_desc,weights=(0, 1, 0))
    #print(score)
    return(score)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
img = image.load_img('/content/drive/MyDrive/COCO_train2014_000000027511.jpg', target_size=(299, 299))


In [ ]:
image_1=encode('/content/drive/MyDrive/COCO_train2014_000000029214.jpg').reshape((1,2048))
print("Greedy:",greedySearch(image_1))
img_caption = greedySearch(image_1)

Greedy: man in blue shirt is working on the street with his hand on his head is standing on the ground with his hand on his head is standing on the ground with his hand on his head is standing on the ground in front of the building with the background of the background of the background is it is on the side of the street with the background and
